In [1]:
#Basic imports

import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

#session Built

spark = SparkSession.builder.appName('Basics_pyspark_df').getOrCreate()

simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [2]:
# groupby on one column

df.groupBy("department").sum("salary").show(truncate=False)

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|Sales     |257000     |
|Finance   |351000     |
|Marketing |171000     |
+----------+-----------+



In [3]:
# groupby on multi-columns

df.groupBy(["department","state"]).sum("salary").show(truncate=False)

+----------+-----+-----------+
|department|state|sum(salary)|
+----------+-----+-----------+
|Sales     |NY   |176000     |
|Sales     |CA   |81000      |
|Finance   |CA   |189000     |
|Finance   |NY   |162000     |
|Marketing |NY   |91000      |
|Marketing |CA   |80000      |
+----------+-----+-----------+



In [7]:
#count
df.groupBy("department").count().show()

#max
df.groupBy("department").max("salary").show()

#min
df.groupBy("department").min("salary").show()

#average
df.groupBy("department").min("bonus").show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+

+----------+-----------+
|department|max(salary)|
+----------+-----------+
|     Sales|      90000|
|   Finance|      99000|
| Marketing|      91000|
+----------+-----------+

+----------+-----------+
|department|min(salary)|
+----------+-----------+
|     Sales|      81000|
|   Finance|      79000|
| Marketing|      80000|
+----------+-----------+

+----------+----------+
|department|min(bonus)|
+----------+----------+
|     Sales|     10000|
|   Finance|     15000|
| Marketing|     18000|
+----------+----------+



In [9]:
#multi-columns and multi-agg

df.groupBy("department","state").sum("salary","bonus").show()

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|     Sales|   NY|     176000|     30000|
|     Sales|   CA|      81000|     23000|
|   Finance|   CA|     189000|     47000|
|   Finance|   NY|     162000|     34000|
| Marketing|   NY|      91000|     21000|
| Marketing|   CA|      80000|     18000|
+----------+-----+-----------+----------+



In [11]:
# using filter/where with groupby    #Note: this agg + where/filter works same as HAVING clause in SQL.

df.groupBy("department").agg(
    sum("salary").alias("sum_salary"),
    avg("salary").alias("avg_salary"),
    sum("bonus").alias("sum_bonus"),
    max("bonus").alias("max_bonus")).where(col("sum_bonus") >= 50000).show(truncate=False)

+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
+----------+----------+-----------------+---------+---------+



### Joins

In [13]:
dept_data = [(1,'Sales','SL01','Mohit'),
             (2,'Finance','FN05','Ravi'),
             (3,'Marketing','MK04','Shekhar')]
dept_col = ['Dept_No','Dept_Name','Dept_Code','Dept_Head']

dept_df = spark.createDataFrame(data=dept_data, schema = dept_col)
dept_df.show()

+-------+---------+---------+---------+
|Dept_No|Dept_Name|Dept_Code|Dept_Head|
+-------+---------+---------+---------+
|      1|    Sales|     SL01|    Mohit|
|      2|  Finance|     FN05|     Ravi|
|      3|Marketing|     MK04|  Shekhar|
+-------+---------+---------+---------+



In [15]:
df.join(dept_df, df.department == dept_df.Dept_Name, 'inner').drop('department').show()

+-------------+-----+------+---+-----+-------+---------+---------+---------+
|employee_name|state|salary|age|bonus|Dept_No|Dept_Name|Dept_Code|Dept_Head|
+-------------+-----+------+---+-----+-------+---------+---------+---------+
|        Maria|   CA| 90000| 24|23000|      2|  Finance|     FN05|     Ravi|
|        Raman|   CA| 99000| 40|24000|      2|  Finance|     FN05|     Ravi|
|        Scott|   NY| 83000| 36|19000|      2|  Finance|     FN05|     Ravi|
|          Jen|   NY| 79000| 53|15000|      2|  Finance|     FN05|     Ravi|
|         Jeff|   CA| 80000| 25|18000|      3|Marketing|     MK04|  Shekhar|
|        Kumar|   NY| 91000| 50|21000|      3|Marketing|     MK04|  Shekhar|
|        James|   NY| 90000| 34|10000|      1|    Sales|     SL01|    Mohit|
|      Michael|   NY| 86000| 56|20000|      1|    Sales|     SL01|    Mohit|
|       Robert|   CA| 81000| 30|23000|      1|    Sales|     SL01|    Mohit|
+-------------+-----+------+---+-----+-------+---------+---------+---------+

### union

Dataframe union() – union() method of the DataFrame is used to merge two DataFrame’s of the same structure/schema. If schemas are not the same it returns an error.


DataFrame unionAll() – unionAll() is deprecated since Spark “2.0.0” version and replaced with union().

In [16]:
dept_data = [(1,'Sales','SL01','Mohit'),
             (2,'Finance','FN05','Ravi'),
             (3,'Marketing','MK04','Shekhar')]
dept_col = ['Dept_No','Dept_Name','Dept_Code','Dept_Head']

dept_df = spark.createDataFrame(data=dept_data, schema = dept_col)
dept_df.show()

dept_data2 = [(1,'Sales','SL01','Mohit'),
             (4,'IT','IT05','Vinod'),
             (5,'Consultancy','CL04','Suresh')]
dept_col2 = ['Dept_No','Dept_Name','Dept_Code','Dept_Head']

dept_df2 = spark.createDataFrame(data=dept_data2, schema = dept_col2)
dept_df2.show()

+-------+---------+---------+---------+
|Dept_No|Dept_Name|Dept_Code|Dept_Head|
+-------+---------+---------+---------+
|      1|    Sales|     SL01|    Mohit|
|      2|  Finance|     FN05|     Ravi|
|      3|Marketing|     MK04|  Shekhar|
+-------+---------+---------+---------+

+-------+-----------+---------+---------+
|Dept_No|  Dept_Name|Dept_Code|Dept_Head|
+-------+-----------+---------+---------+
|      1|      Sales|     SL01|    Mohit|
|      4|         IT|     IT05|    Vinod|
|      5|Consultancy|     CL04|   Suresh|
+-------+-----------+---------+---------+



In [17]:
dept_df.union(dept_df2).show()                     # if you notice union may have duplicates

+-------+-----------+---------+---------+
|Dept_No|  Dept_Name|Dept_Code|Dept_Head|
+-------+-----------+---------+---------+
|      1|      Sales|     SL01|    Mohit|
|      2|    Finance|     FN05|     Ravi|
|      3|  Marketing|     MK04|  Shekhar|
|      1|      Sales|     SL01|    Mohit|
|      4|         IT|     IT05|    Vinod|
|      5|Consultancy|     CL04|   Suresh|
+-------+-----------+---------+---------+



In [18]:
#to drop duplicates

dept_df.union(dept_df2).distinct().show() 

+-------+-----------+---------+---------+
|Dept_No|  Dept_Name|Dept_Code|Dept_Head|
+-------+-----------+---------+---------+
|      1|      Sales|     SL01|    Mohit|
|      2|    Finance|     FN05|     Ravi|
|      3|  Marketing|     MK04|  Shekhar|
|      4|         IT|     IT05|    Vinod|
|      5|Consultancy|     CL04|   Suresh|
+-------+-----------+---------+---------+



In [21]:
df1 = spark.createDataFrame([[1,'Ravi']], ["id","name"])
df2 = spark.createDataFrame([['Ram',2]], ["name","id"])

# Note let's check the flaw in union
df1.union(df2).show()

+---+----+
| id|name|
+---+----+
|  1|Ravi|
|Ram|   2|
+---+----+



The difference between unionByName() function and union() is that this function
resolves columns by name (not by position). In other words, unionByName() is used to merge two DataFrames by column names instead of by position.

In [20]:
# unionByName

# Using allowMissingColumns
df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.printSchema
df3.show()

+---+----+
| id|name|
+---+----+
|  1|Ravi|
|  2| Ram|
+---+----+



In [19]:
# Create DataFrames with different column names
df1 = spark.createDataFrame([[5, 2, 6]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[6, 7, 3]], ["col1", "col2", "col3"])

# Using allowMissingColumns
df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.printSchema
df3.show()

+----+----+----+----+
|col0|col1|col2|col3|
+----+----+----+----+
|   5|   2|   6|null|
|null|   6|   7|   3|
+----+----+----+----+

